In [42]:
# imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier,RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import warnings
warnings.filterwarnings('ignore')

In [43]:
glass_df = pd.read_csv(r'..\Cases\Glass Identification\Glass.csv')
glass_df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,building_windows_float_processed


In [44]:
X= glass_df.drop(columns=['Type'])
y= glass_df['Type']

scaler=StandardScaler()

le= LabelEncoder()
y= le.fit_transform(y)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

## Using LGB as final estimator 

### Without passthrough

In [45]:
lda=LinearDiscriminantAnalysis()
lr=LogisticRegression(random_state=25)

svc=SVC(kernel='rbf',probability=True,random_state=25)
sm_pipe=Pipeline([("Scalar",scaler),("svm",svc)])

dtree=DecisionTreeClassifier(random_state=25)
lgb=LGBMClassifier(random_state=25)
random_forest=RandomForestClassifier(random_state=25)
stack_1=StackingClassifier([('Linear discriminant',lda),('LinearRegression',lr),("SVC",sm_pipe),('DecisionTree',dtree)],final_estimator=lgb)

In [46]:
stack_1.fit(X_train,y_train)
y_pred_proba_1=stack_1.predict_proba(X_test)
log_loss(y_test,y_pred_proba_1)

1.1106425457558524

### With passthrough

In [47]:
stack_2=StackingClassifier([('Linear discriminant',lda),('LinearRegression',lr),("SVC",sm_pipe),('DecisionTree',dtree)],final_estimator=lgb,passthrough=True)
stack_2.fit(X_train,y_train)
y_pred_proba_2=stack_2.predict_proba(X_test)
log_loss(y_test,y_pred_proba_2)

0.7725145885617061

## With randomForest as final estimator

### Without pass through

In [48]:
stack_3=StackingClassifier([('Linear discriminant',lda),('LinearRegression',lr),("SVC",sm_pipe),('DecisionTree',dtree)],final_estimator=random_forest)
stack_3.fit(X_train,y_train)
y_pred_proba_3=stack_3.predict_proba(X_test)
log_loss(y_test,y_pred_proba_3)

0.823414723313914

In [49]:
stack_4=StackingClassifier([('Linear discriminant',lda),('LinearRegression',lr),("SVC",sm_pipe),('DecisionTree',dtree)],final_estimator=random_forest,passthrough=True)
stack_4.fit(X_train,y_train)
y_pred_proba_4=stack_4.predict_proba(X_test)
log_loss(y_test,y_pred_proba_4)

0.6834076239625183